In [69]:
import sys

import pandas as pd
import numpy as np
df = pd.read_excel("Data/S1%20File.xlsx")
#Rimozione colonne inutili e valori nulli
df.drop(columns=["Average weekday", "Average weekend day"], inplace=True)
df.dropna(inplace=True)

#Lista ordine delle colonne
columns_order = df.columns

#Cambio del valore in Genere 1:Maschio 0:Femmina
df["Gender"] = np.where(df["Gender"] == 2, 1,0)

#Normalizzazione
gender = df["Gender"]
id_no = df["ID No"]
df.drop(columns=["Gender", "ID No"], inplace=True)

df = (df - df.mean())/df.std()
df["Gender"] = gender
#df["ID No"] = id_no

#df = df[columns_order]
df.reset_index(drop=True, inplace=True)

In [72]:
from random import sample, seed
#random forest per livello ansia, senza depressione.
y_depression = df["Total Depression score"]
df_smaller = df.drop(columns=["Total anxiety score", "Total Depression score"], inplace = False)

seed(33)
training_idx_depression = sample(range(0,df.shape[0]),25)
validation_idx_depression = sample(range(0,df.shape[0]),20)
test_idx_depression = sample(range(0,df.shape[0]),20)

training_depression = df_smaller.iloc[training_idx_depression]
validation_depression = df_smaller.iloc[validation_idx_depression]
test_depression = df_smaller.iloc[test_idx_depression]

#y_depression.iloc[training_idx_depression]
training_depression

,Age,Height (m),Weight (kgs),BMI,working hours per week,RP,RE,BP,MH,VT,...,Time standing (h),Time stepping (h),Step count,Sit to stand transitions,Sitting only.1,Time standing (h).1,Time stepping (h).1,Step count.1,Sit to stand transitions.1,Gender
21,-1.329718,-0.834830,-0.129503,-0.416880,-0.375835,0.430939,0.298681,0.574851,1.192380,1.876699,...,-1.746711,-0.932873,-0.664946,-1.073608,0.989088,-0.725416,1.152502,1.844818,-0.555733,0
29,0.794954,-0.833239,1.611489,0.356721,1.625200,0.430939,0.298681,-0.759624,0.393362,0.474568,...,0.103399,0.429447,0.292331,-0.721856,1.784979,-0.844742,-1.065254,-1.007798,-0.381107,0
52,0.539994,1.279491,0.697468,0.299015,0.180008,0.430939,0.298681,0.574851,-1.204672,-0.927564,...,0.887918,-0.450296,-0.439387,-0.975117,-0.804291,1.829723,-0.764273,-0.891048,-1.104558,1
53,1.389863,1.053105,-1.156688,-0.346845,0.180008,-1.043326,-1.858457,-0.759624,-2.003689,-2.329696,...,0.340359,0.015199,-0.121687,0.180975,-1.085687,0.799885,0.827759,0.511705,0.966008,1
17,-1.584679,-0.833484,0.566894,-0.525563,-3.710894,0.430939,0.298681,0.574851,-0.405655,0.474568,...,0.018822,-0.819275,-0.858186,-0.749996,0.447295,-0.121002,0.677269,0.650985,-1.329077,0
30,-1.584679,-0.833239,1.002142,-0.243719,0.735851,0.430939,0.298681,0.574851,-1.204672,-0.927564,...,-0.853555,-1.201920,-1.193623,-0.721856,1.742980,-1.262385,-1.841469,-1.748676,-0.954878,0
41,0.879941,1.138764,-0.268782,0.181586,0.180008,0.430939,0.298681,-0.759624,-0.405655,0.474568,...,-0.832775,-0.655284,-0.547625,-1.369080,-0.947089,-0.408942,0.083227,-0.084041,-0.979825,1
56,-0.479849,1.181594,0.497254,0.798088,0.180008,0.430939,0.298681,0.574851,-1.204672,0.474568,...,-1.836027,-0.911520,-0.658837,-0.893823,-0.482994,-0.372625,-0.526656,-0.827845,0.467077,1
34,-0.309875,-0.834732,0.044596,-0.284415,0.402345,0.430939,0.298681,0.574851,-1.204672,1.876699,...,0.194537,-0.095837,-0.143201,0.849304,2.538870,-1.241632,-0.273198,0.355453,-0.231428,0
33,-0.309875,-0.835075,-1.174098,-1.471461,0.068839,0.430939,0.298681,0.574851,-0.405655,0.474568,...,1.760903,0.637282,0.138389,1.185423,0.705592,0.478223,-0.542497,-0.874077,1.514833,1


In [76]:
#crea la random forest per la depressione
from sklearn.ensemble import RandomForestRegressor 
#TODO fare il validation per parametri B ed MTRY, questa è solo una prova...
min_validation_error = sys.maxsize
min_model = None
list_n_trees = [10,20,30,60,100,125,150,200,300,400]
for i in range(0,len(list_n_trees)):
    rf = RandomForestRegressor(n_estimators=list_n_trees[i])
    rf.fit(training_depression, y_depression.iloc[training_idx_depression])
    y_predict_depression = rf.predict(validation_depression)
    #Calcolo del validation error...
    MSE = np.mean((y_depression.iloc[validation_idx_depression]-y_predict_depression)**2)
    print(MSE)
    if min_validation_error > MSE:
        min_model = rf
        min_validation_error = MSE




0.7235547355473553
0.6694880073800736
0.5661404947382805
0.5238864971983052
0.5368880073800737
0.5656678966789668
0.5288489544895449
0.5290292435424356
0.5135960297936316
0.5491715982472328


In [66]:
from sklearn.ensemble import RandomForestRegressor 
from random import sample, seed
#random forest per livello depressione, senza ansia.
y_anxiety = df["Total anxiety score"]
df_anxiety = df.drop(columns=["Total anxiety score", "Total Depression score"], inplace = False)

seed(333)
#TODO riprendere il test set precedente
training_idx_anxiety = sample(range(0,df.shape[0]),45)
test_idx_anxiety = sample(range(0,df.shape[0]),20)

training_anxiety = df_smaller.iloc[training_idx_anxiety]
test_idx_anxiety = df_smaller.iloc[test_idx_anxiety]

